In [ ]:
import pandas as pd
import numpy as np
import copy
from time import time

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

Function that calculates distance between two points using their latitude and longitude

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def get_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance, 2)

Function that transforms hour(0-23) into part of the day

In [ ]:
def get_time(time_str):
#     v = int(data1["starttime"][i][11:13])
    int_time = int(time_str[11:13])
    if 0 <= int_time and int_time < 6:
        return "night"
    if 6 <= int_time and int_time < 12:
        return "morning"
    if 12 <= int_time and int_time < 18:
        return "afternoon"
    if 18 <= int_time and int_time <= 23:
        return "evening"

Data import

In [ ]:
data = pd.read_csv('data/2018/201806-citibike-tripdata.csv')

In [ ]:
from sklearn.utils import shuffle

cc=time()
data = shuffle(data)
print(time() - cc)

In [ ]:
print(len(data))

In [ ]:
data1 = copy.deepcopy(data)
# data1 = data[0:1000]

In [ ]:
print(len(data1))

In [ ]:
data1.head(5)

Adding new colums - distance and part of the day

In [ ]:
from time import time
new_times = [" "] * len(data) 
distances = [0] * len(data)
print(len(new_times))
c=time()

for i, row in data1.iterrows():
#     print(i)
    new_times[i] = get_time(row["starttime"])
    distances[i] = get_distance(row["start station latitude"], row["start station longitude"], row["end station latitude"], row["end station longitude"])
    
data1["time"] = pd.Series(new_times)
data1["distance"] = pd.Series(distances)

print(time() - c)

Removing not required colums

In [ ]:
del data1["starttime"]
del data1["stoptime"]
del data1["start station id"]
del data1["start station name"]
del data1["end station id"]
del data1["end station name"]
del data1["bikeid"]
del data1["start station latitude"]
del data1["start station longitude"]
del data1["end station latitude"]
del data1["end station longitude"]

In [ ]:
data1.head(5)

Change indexes of rows from shuffled to ordered

In [ ]:
data1 = data1.reset_index(drop=True)
data1.head(5)

In [ ]:
data1.to_csv("./data/2018/processed.csv", index=False)

Splitting data into train/test and x/y parts for future linear regression

In [ ]:
k = int(0.8*len(data1))
print(k)
x_train = data1["tripduration"][:k]
x_test = data1["tripduration"][k:]
y_train = data1["distance"][:k]
y_test = data1["distance"][k:]

In [ ]:
print(len(x_train), len(x_test))

Initialize linear regression

In [ ]:
model = LinearRegression()

model.fit(list(map(lambda x:[x],x_train)), y_train)

In [ ]:
predict = model.predict(list(map(lambda x:[x],x_test)))

LR results

In [ ]:
model.score(list(map(lambda x:[x],x_train)), y_train)

In [ ]:
model.score(list(map(lambda x:[x],x_test)), y_test)